# Задачи к Лекции 6

Дан файл "Anuran_Calls_(MFCCs).zip", в котором содержится обработка звуков издаваемых различными видами лягушек. В качестве признаков выступают мел-кепстральные коэффициенты __[MFCC](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum)__

**Чтение данных**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%matplotlib inline
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

z = zipfile.ZipFile("/content/drive/MyDrive/ML/ml2021p/data/Anuran_Calls_(MFCCs).zip")
df = pd.read_csv(z.open("Frogs_MFCCs.csv"))

df.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,MFCCs_11,MFCCs_12,MFCCs_13,MFCCs_14,MFCCs_15,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,0.188654,-0.075622,-0.156436,0.082245,0.135752,-0.024017,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,0.270958,-0.095004,-0.254341,0.022786,0.163320,0.012022,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,0.266064,-0.072827,-0.237384,0.050791,0.207338,0.083536,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,0.267279,-0.162258,-0.317084,-0.011567,0.100413,-0.050224,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,0.332695,-0.100749,-0.298524,0.037439,0.219153,0.062837,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1


**Получение различной информации**

In [3]:
df.shape

(7195, 26)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7195 entries, 0 to 7194
Data columns (total 26 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MFCCs_ 1  7195 non-null   float64
 1   MFCCs_ 2  7195 non-null   float64
 2   MFCCs_ 3  7195 non-null   float64
 3   MFCCs_ 4  7195 non-null   float64
 4   MFCCs_ 5  7195 non-null   float64
 5   MFCCs_ 6  7195 non-null   float64
 6   MFCCs_ 7  7195 non-null   float64
 7   MFCCs_ 8  7195 non-null   float64
 8   MFCCs_ 9  7195 non-null   float64
 9   MFCCs_10  7195 non-null   float64
 10  MFCCs_11  7195 non-null   float64
 11  MFCCs_12  7195 non-null   float64
 12  MFCCs_13  7195 non-null   float64
 13  MFCCs_14  7195 non-null   float64
 14  MFCCs_15  7195 non-null   float64
 15  MFCCs_16  7195 non-null   float64
 16  MFCCs_17  7195 non-null   float64
 17  MFCCs_18  7195 non-null   float64
 18  MFCCs_19  7195 non-null   float64
 19  MFCCs_20  7195 non-null   float64
 20  MFCCs_21  7195 non-null   floa

In [5]:
df.describe()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,MFCCs_11,MFCCs_12,MFCCs_13,MFCCs_14,MFCCs_15,MFCCs_16,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,RecordID
count,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000
mean,0.989885,0.323584,0.311224,0.445997,0.127046,0.097939,-0.001397,-0.000370,0.128213,0.055998,-0.115682,0.043371,0.150945,-0.039244,-0.101748,0.042062,0.088680,0.007755,-0.049474,-0.053244,0.037313,0.087567,25.220014
std,0.069016,0.218653,0.263527,0.160328,0.162722,0.120412,0.171404,0.116302,0.179008,0.127099,0.186792,0.155983,0.206880,0.152515,0.187618,0.119915,0.138055,0.084733,0.082546,0.094181,0.079470,0.123442,13.214399
min,-0.251179,-0.673025,-0.436028,-0.472676,-0.636012,-0.410417,-0.538982,-0.576506,-0.587313,-0.952266,-0.901989,-0.799441,-0.644116,-0.590380,-0.717156,-0.498675,-0.421480,-0.759322,-0.680745,-0.361649,-0.430812,-0.379304,1.000000
25%,1.000000,0.165945,0.138445,0.336737,0.051717,0.012581,-0.125737,-0.063109,0.004648,-0.001132,-0.269860,-0.033931,-0.002859,-0.132980,-0.255929,-0.019549,-0.001764,-0.042122,-0.106079,-0.120971,-0.017620,0.000533,15.000000
50%,1.000000,0.302184,0.274626,0.481463,0.161361,0.072079,-0.052630,0.013265,0.189317,0.063478,-0.153322,0.051054,0.196921,-0.050715,-0.143259,0.041081,0.112769,0.011820,-0.052626,-0.055180,0.031274,0.105373,22.000000
75%,1.000000,0.466566,0.430695,0.559861,0.222592,0.175957,0.085580,0.075108,0.265395,0.117725,0.026689,0.132432,0.324589,0.039157,0.017348,0.107046,0.201932,0.061889,0.006321,0.001342,0.089619,0.194819,37.000000
max,1.000000,1.000000,1.000000,1.000000,0.752246,0.964240,1.000000,0.551762,0.738033,0.522768,0.523033,0.690889,0.945710,0.575749,0.668924,0.670700,0.681157,0.614064,0.574209,0.467831,0.389797,0.432207,60.000000


In [6]:
df["Family"].value_counts()

Leptodactylidae    4420
Hylidae            2165
Dendrobatidae       542
Bufonidae            68
Name: Family, dtype: int64

**1. Провести интервальную оценку среднего значения признака "MFCCs_ 2" с помощью бутстрапа с точностью 95%.**

In [7]:
def bootstrap(data, n_samples):
    ind = np.random.randint(0, len(data), (n_samples, len(data)))
    return data[ind]

def interval(stat, alpha):
    return np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])

In [8]:
np.random.seed(17)

MFC_2 = df['MFCCs_ 2'].values

MFC_mean_score = [np.mean(sample) for sample in bootstrap(MFC_2, 1000)]
print(f'mean interval: {interval(MFC_mean_score, 0.05)}')

mean interval: [0.31856885 0.32847859]


**2. Построить лес случайных деревьев, для определения семейства Dendrobatidae. Вывести 5 самых важных признаков. Сравнить результат с логистической регрессией.**

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [10]:
X = df.drop(df.columns[-4:], axis=1)
y = (df["Family"] == 'Dendrobatidae').astype('int')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

In [11]:
pipel1 = make_pipeline(StandardScaler(),
                      RandomForestClassifier(n_estimators=50, n_jobs=-1))
pipel2 = make_pipeline(StandardScaler(),
                     LogisticRegression(C=100))

In [12]:
res = pipel1.fit(X_train, y_train)
ROC_rf = roc_auc_score(y_test, res.predict_proba(X_test)[:, 1])

res1 = pipel2.fit(X_train, y_train)
ROC_logit = roc_auc_score(y_test, res1.predict_proba(X_test)[:, 1])

print(f'rf: {ROC_rf}\nlogit: {ROC_logit}')

rf: 0.9999631329087001
logit: 0.996679125852906


In [13]:
feature_importances = pd.DataFrame(zip(X.columns, res[1].feature_importances_),
                                  columns=["Name", "Importance"])
feature_importances.sort_values(by="Importance", ascending=False)

,Name,Importance
14,MFCCs_15,0.106966
2,MFCCs_ 3,0.104298
16,MFCCs_17,0.098010
15,MFCCs_16,0.079529
4,MFCCs_ 5,0.075616
13,MFCCs_14,0.058165
1,MFCCs_ 2,0.048401
6,MFCCs_ 7,0.041525
20,MFCCs_21,0.039178
9,MFCCs_10,0.037002


**3. Использовать бэггинг для обучения 100 логистических регрессий для определения семейства Dendrobatidae. Использовать RandomizedSearchCV для того, чтобы не перебирать все возможные комбинации параметров. Выбрать для RandomizedSearchCV - 20 итераций.**

In [28]:
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

parameters = {
    'max_features': [5, 10, 15],    # кол-во случайно отбираемых признаков
    'max_samples': [0.5, 0.7, 0.9], # процент выборки для обучения 
    'base_estimator__C': np.logspace(-4, 5, 10), # регуляризация
}

pipel3 = make_pipeline(StandardScaler(),
                       RandomizedSearchCV(BaggingClassifier(LogisticRegression(), n_estimators=100, n_jobs=-1), parameters, n_iter=20, n_jobs=-1))

res = pipel3.fit(X_train, y_train)
print('ROC:', roc_auc_score(y_test, res.predict_proba(X_test)[:, 1]))

ROC: 0.9963529938914066


**4. Организовать стекинг моделей (любой из вариантов) для определения семейства Dendrobatidae. Для этого выбрать базовые модели и выбрать мета-модель (самостоятельно). Определить постепенным добавлением, какие модели улучшают результат классификации, какие ухудшают.**

In [27]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

base_models = [('random_forest',
               RandomForestClassifier(n_estimators=50)),
              ('knn', KNeighborsClassifier(n_neighbors=11)),
               ('svr', LinearSVC(random_state=42))]

meta_model = LogisticRegression(C=100)
stacking_model = StackingClassifier(estimators=base_models,
                                    final_estimator=meta_model)

stacking_model.fit(X_train, y_train)
print('ROC:', roc_auc_score(y_test, stacking_model.predict_proba(X_test)[:, 1]))

ROC: 1.0
